# <Strong>Mount Drive</Strong>

In [ ]:
from google.colab import drive 

drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


## <strong>NLP_Term - NSMC</strong>

###1. LSTM


In [ ]:
!python3 -m pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 12.8MB/s 
     |████████████████████████████████| 3.6MB 56.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

In [ ]:
train_data = pd.read_table('/content/gdrive/My Drive/NLP_Term/NSMC/Data/ratings_train.txt')
test_data = pd.read_table('/content/gdrive/My Drive/NLP_Term/NSMC/Data/ratings_test.txt')
stop_word = pd.read_table('/content/gdrive/My Drive/NLP_Term/NSMC/Data/stopword.txt')

In [ ]:
train_data['document'].nunique(), train_data['label'].nunique()

(146182, 2)

In [ ]:
train_data = train_data.dropna(how = 'any')
print(train_data.isnull().values.any())

False


In [ ]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","")

In [ ]:
train_data = train_data.dropna(how = 'any')
print(train_data.isnull().values.any())

False


In [ ]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거

In [ ]:
stopwords = stop_word['word']
print(stopwords[:5])

0    이
1    있
2    하
3    것
4    들
Name: word, dtype: object


In [ ]:
okt = Okt()
x_train = []
cnt = 0
for sentence in train_data['document']:
    cnt = cnt + 1
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    if len(temp_X) == 0:
        temp_X = ["좋다"]
    x_train.append(temp_X)
    if cnt % 10000 == 0:
        print(cnt)

x_test = []
for sentence in test_data['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    if len(temp_X) == 0:
        temp_X = ["좋다"]
    x_test.append(temp_X)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, GlobalAveragePooling1D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers, optimizers 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

total_cnt = len(tokenizer.word_index) + 1 # 단어의 수

history = []
for word_num in range(5000, total_cnt, 5000):
    train_x = x_train[:]
    test_x = x_test[:]
    tokenizer = Tokenizer(word_num)
    tokenizer.fit_on_texts(train_x)
    train_x = tokenizer.texts_to_sequences(train_x)
    test_x = tokenizer.texts_to_sequences(test_x)

    train_y = np.array(train_data['label'])
    test_y = np.array(test_data['label'])
    
    drop_train = [index for index, sentence in enumerate(x_train) if len(sentence) < 1]
    drop_test = [index for index, sentence in enumerate(x_test) if len(sentence) < 1]
    
    train_x = np.delete(train_x, drop_train, axis = 0)
    test_x = np.delete(test_x, drop_test, axis = 0)

    train_y = np.delete(train_y, drop_train, axis = 0)
    test_y = np.delete(test_y, drop_test, axis = 0)

    model_num = int(word_num / 5000)
    print("%d" %(tokenizer.word_index["좋다"]))
    max_len = 30
    train_x = pad_sequences(train_x, maxlen = max_len)
    test_x = pad_sequences(test_x, maxlen = max_len)
    model = Sequential()
    model.add(Embedding(word_num, 300))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(LSTM(256))
    model.add(Dense(1, activation='sigmoid'))
    #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
    mc = ModelCheckpoint('best_model' + str(model_num) + ".h5", monitor='val_acc', mode='max', verbose=1, save_best_only=True)
    model.compile(optimizer='adam' , loss='binary_crossentropy', metrics=['acc'])
    history.append(model.fit(train_x, train_y, epochs=15, batch_size=256, validation_data = (test_x, test_y), callbacks = [mc]))
    model.save("/content/gdrive/My Drive/NLP_Term/NSMC/Models/best_model" + str(model_num) + ".h5")

16
Epoch 1/15
585/586 [============================>.] - ETA: 0s - loss: 0.4058 - acc: 0.8129
Epoch 00001: val_acc improved from -inf to 0.83293, saving model to best_model1.h5
586/586 [==============================] - 19s 32ms/step - loss: 0.4056 - acc: 0.8130 - val_loss: 0.3730 - val_acc: 0.8329
Epoch 2/15
585/586 [============================>.] - ETA: 0s - loss: 0.3566 - acc: 0.8416
Epoch 00002: val_acc improved from 0.83293 to 0.83954, saving model to best_model1.h5
586/586 [==============================] - 18s 31ms/step - loss: 0.3565 - acc: 0.8416 - val_loss: 0.3566 - val_acc: 0.8395
Epoch 3/15
585/586 [============================>.] - ETA: 0s - loss: 0.3370 - acc: 0.8492
Epoch 00003: val_acc improved from 0.83954 to 0.84078, saving model to best_model1.h5
586/586 [==============================] - 18s 31ms/step - loss: 0.3370 - acc: 0.8491 - val_loss: 0.3548 - val_acc: 0.8408
Epoch 4/15
585/586 [============================>.] - ETA: 0s - loss: 0.3232 - acc: 0.8560
Epoch 000

In [ ]:
score_data = pd.read_csv("/content/gdrive/My Drive/NLP_Term/NSMC/Data/ko_data.csv", encoding='cp949')
score_data

,Id,Sentence
0,0,정말 많이 울었던 영화입니다.
1,1,시간 낭비예요.
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.
...,...,...
11182,11182,이 영화를 커플에게 추천합니다. 영화관에 가다보면 평생 잊지 못할 추억이 하나 생길...
11183,11183,심심__ 그냥 한효주 cf
11184,11184,공감해서 눈물나는 영화. 안 보신분들이 전부 제가 울었다고 하면 의아해하실텐데 보면...
11185,11185,오토바이 신은 최고네요.


In [ ]:
score_data['Sentence'] = score_data['Sentence'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","")

In [ ]:
x_score = []
for sentence in score_data['Sentence']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    if len(temp_X) == 0:
        temp_X = ["좋다"]
    x_score.append(temp_X)

In [ ]:
results = []
for word_num in range(5000, total_cnt, 5000):
    train_x = x_train[:]
    score_x = x_score[:]
    tokenizer = Tokenizer(word_num)
    tokenizer.fit_on_texts(train_x)
    train_x = tokenizer.texts_to_sequences(train_x)
    score_x = tokenizer.texts_to_sequences(score_x)
    
    drop_score = [index for index, sentence in enumerate(x_test) if len(sentence) < 1]
    for idx in drop_score:
        score_x[idx] = [tokenizer.word_index["좋다"]]
    max_len = 30
    score_x = pad_sequences(score_x, maxlen = max_len)

    model_num = int(word_num / 5000)
    loaded_model = load_model("/content/gdrive/My Drive/NLP_Term/NSMC/Models/best_model" + str(model_num) + ".h5")
    results.append(loaded_model.predict(score_x))

In [ ]:
results = np.rint(results)
results.astype(np.int)
predicted = []
for i in range(len(results)):
    predicted.append([])
    for j in range(len(results[i])):
        predicted[i].append(int(results[i][j][0]))

In [ ]:
for i in range(len(results)):
    new_df = pd.DataFrame({"Id": [i for i in range(len(predicted[i]))], "Predicted": predicted[i]})
    new_df.to_csv("/content/gdrive/My Drive/NLP_Term/NSMC/result/solution" + str(i) + ".csv", index=False)

## <strong>NLP_Term - Friends</strong>

### 1. LSTM

In [ ]:
import os
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144439 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.22-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.22-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.22-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
import numpy as np 
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
os.chdir('gdrive/My Drive/NLP_Term/Friends')

FileNotFoundError: ignored

In [ ]:
#Reading the csv files

data = pd.read_csv('Data/friends_train.csv')
pd.set_option('display.max_colwidth',-1)
data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


,idx,sentence,emotion
0,1,also i was the point person on my company ' s transition from the kl - 5 to gr - 6 system .,neutral
1,2,you must ' ve had your hands full .,neutral
2,3,that i did . that i did .,neutral
3,4,so let ' s talk a little bit about your duties .,neutral
4,5,my duties ? all right .,surprise


In [ ]:
import nltk
nltk.download('stopwords')

nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
sentence2 = []
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer

stopwords_set = list(stopwords.words("english"))
for i in data['sentence']:
  replaced = re.sub(r"{.*?}", '', i)
  replaced = re.sub('[^a-zA-Z]', ' ', replaced)
  sentence2.append(replaced)

sentence3 = []  
import re
for j in sentence2:
  query = j
  stopwords = stopwords_set

  resultwords  = [word for word in re.split("\W+",query) if word.lower() not in stopwords]
  lemmatizer = WordNetLemmatizer()
  stemmer_words = [lemmatizer.lemmatize(word) for word in resultwords]

  result = ' '.join(stemmer_words)
  #print(result)
  sentence3.append(result)
print(sentence3)

['also point person company transition kl gr system ', 'must hand full ', '', 'let talk little bit duty ', 'duty right ', 'heading whole division lot duty ', 'see ', 'perhaps people dump certain amount ', 'good know ', 'go detail', 'beg ', 'right definite answer monday think say confidence fit well ', 'really ', 'absolutely relax great ', 'waitress went last month ', 'know forget ', 'talking ', 'actually know', 'ok ', 'right well ', 'gonna see get room night ', 'see later ', 'yeah sure ', 'hey mon ', 'hey hey hey wanna hear something suck ', 'ever ', 'chris say closing bar ', 'way ', 'yeah apparently turning kinda coffee place ', 'coffee gonna hang ', 'got ', 'get beer ', 'hey pick roommate ', 'betcha ', 'italian guy ', 'um mm yeah right ', 'oh god oh god poor monica ', '', '', 'wrote poem ', 'look vessel empty nothing inside ', 'touched seem emptier still ', 'think monica empty empty vase ', 'really think meant ', 'oh totally oh god oh seemed happy ', 'done ', 'hey ', 'hi ', '', 'ah k

In [ ]:
import string

In [ ]:
#Toekenizing the words

num_words = 1000
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
print(tokenizer)
X=tokenizer.fit_on_texts(sentence3)
print(X)
X = tokenizer.texts_to_sequences(sentence3)


word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 100
X = pad_sequences(X, maxlen=max_length_of_text)

None
Found 4953 unique tokens.


In [ ]:
X[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0, 280, 460, 308, 807, 713], dtype=int32)

In [ ]:
y=data['emotion']

In [ ]:
!pip install category_encoders

     |████████████████████████████████| 81kB 5.3MB/s 


In [ ]:
#Onehot encoding all the 7 labels

import category_encoders as ce
le =  ce.OneHotEncoder(handle_unknown="ignore")
#enc= OneHotEncoder(handle_unknown='ignore')
#enc.fit(y)
img_label=np.array(y)
enc=le.fit_transform(img_label)
#image_labels_enc=img_label.reshape(-1,1)
"""enc = OneHotEncoder()
img_fit=enc.fit(image_labels_enc)
img_encode=enc.transform(y).toarray()"""

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


'enc = OneHotEncoder()\nimg_fit=enc.fit(image_labels_enc)\nimg_encode=enc.transform(y).toarray()'

In [ ]:
enc

,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8
0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
10556,1,0,0,0,0,0,0,0
10557,0,0,0,1,0,0,0,0
10558,0,1,0,0,0,0,0,0
10559,1,0,0,0,0,0,0,0


In [ ]:
X_train, Xvalid, y_train, yvalid = train_test_split(X,enc, test_size = 0.2, random_state = 42)
print(X_train.shape,y_train.shape)
#print(X_test.shape,y_test.shape)

(8448, 100) (8448, 8)


In [ ]:
#LSTM model

embed_dim = 1024
lstm_out = 256
batch_size = 64

inputs = Input((max_length_of_text,))
x = Embedding(num_words, embed_dim)(inputs)
x = Bidirectional(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.3))(x)
x = Dense(8,activation='sigmoid')(x)
model = Model(inputs, x)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 1024)         1024000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               2623488   
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 4104      
Total params: 3,651,592
Trainable params: 3,651,592
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional

model = Sequential()
model.add(Embedding(num_words, embed_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.7))
model.add(LSTM(256, dropout=0.6, recurrent_dropout=0.6))
model.add(Dense(8, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 1024)         1024000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 100, 1024)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               1311744   
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 2056      
Total params: 2,337,800
Trainable params: 2,337,800
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['categorical_accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stops = EarlyStopping(patience=3, monitor='val_acc',min_delta=0.001)
checkpointer = ModelCheckpoint(filepath='weights4.best.eda.hdf5', verbose=1, save_best_only=True)

In [ ]:
model.fit(X_train, y_train,validation_data=(Xvalid, yvalid) ,batch_size = 64, epochs = 20,callbacks=[checkpointer], verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8448 samples, validate on 2113 samples
Epoch 1/20
8448/8448 [==============================] - 31s 4ms/step - loss: 1.6176 - categorical_accuracy: 0.4419 - val_loss: 1.5119 - val_categorical_accuracy: 0.4614

Epoch 00001: val_loss improved from inf to 1.51186, saving model to weights4.best.eda.hdf5
Epoch 2/20
8448/8448 [==============================] - 29s 3ms/step - loss: 1.5132 - categorical_accuracy: 0.4695 - val_loss: 1.4577 - val_categorical_accuracy: 0.4979

Epoch 00002: val_loss improved from 1.51186 to 1.45766, saving model to weights4.best.eda.hdf5
Epoch 3/20
8448/8448 [==============================] - 28s 3ms/step - loss: 1.4395 - categorical_accuracy: 0.4982 - val_loss: 1.4373 - val_categorical_accuracy: 0.4879

Epoch 00003: val_loss improved from 1.45766 to 1.43729, saving model to weights4.best.eda.hdf5
Epoch 4/20
8448/8448 [==============================] - 28s 3ms/step - loss: 1.3896 - categorical_accuracy: 0.5047 - val_loss: 1.4371 - val_categorical_accuracy:

In [ ]:
model.save('nnfl4.eda.hdf5')

In [ ]:
score,acc = model.evaluate(Xvalid, yvalid, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

2113/2113 [==============================] - 1s 322us/step
Score: 1.64
Validation Accuracy: 0.47


In [ ]:
#Loading the test data
data2 = pd.read_csv('Data/en_data.csv', encoding='CP949')
#data2 = pd.read_csv('Data/friends_test.csv', encoding='CP949')
data2.head()

,id,utterance
0,0,"alright , whadyou do with him ?"
1,1,oh ! you ' re awake !
2,2,then you gotta come clean with ma ! this is not right !
3,3,"yeah , but this is"
4,4,i don ' t wanna hear it ! now go to my room !


In [ ]:

#Preprocessing the test data

from keras.models import load_model
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
import numpy as np
from nltk.corpus import stopwords

nltk.download('punkt')

data = pd.read_csv('Data/en_data.csv', encoding='CP949')
#data = pd.read_csv('Data/friends_test.csv', encoding='CP949')
sentence = data['utterance']

stopwords_set = list(stopwords.words("english"))

sentence2 = []
for i in sentence:
  replaced = re.sub(r"{.*?}", '', i)
  replaced = re.sub('[^a-zA-Z]', ' ', replaced)
  sentence2.append(replaced)

sentence3 = []
for j in sentence2:
  query = j
  stopwords = stopwords_set
  tokens = [word for sent in nltk.sent_tokenize(str(query))for word in nltk.word_tokenize(sent)]
  resultwords  = [word for word in re.split("\W+",str(tokens)) if word.lower() not in stopwords]
  lemmatizer = WordNetLemmatizer()
  stemmer_words = [lemmatizer.lemmatize(word) for word in resultwords]
  result = ' '.join(stemmer_words)
  sentence3.append(result)

num_words = 500
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,split=' ')
tokenizer.fit_on_texts(sentence3)
X = tokenizer.texts_to_sequences(sentence3)

word_index = tokenizer.word_index

max_length_of_text = 100
X = pad_sequences(X, maxlen=max_length_of_text)

#Loading the model.Write your own model name here
model = load_model('nnfl4.eda.hdf5')
results = model.predict(X)

results = np.argmax(results, axis=1)

predictions = []

conv = { 0: 'neutral', 
         1: 'joy', 
         2: 'sadness', 
         3: 'anger', 
         4: 'fear', 
         5: 'surprise', 
         6: 'disgust', 
         7: 'non-neutral'}

for i in results:
	predictions.append(conv[i])

'''
k = 0
cnt = 0
total = data['emotion'].size
for i in data['emotion']:
  if predictions[k] == i: 
    cnt = cnt+1
  k = k+1
print(cnt)
print(total)
'''

#Converting the solution into a CSV file
new_df = pd.DataFrame({"Id": [i for i in range(len(predictions))], "Predicted": predictions})
new_df.to_csv("sample.csv", index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


### 2. BERT

In [ ]:
!pip install transformers

     |████████████████████████████████| 675kB 10.6MB/s 
     |████████████████████████████████| 890kB 48.4MB/s 
     |████████████████████████████████| 3.8MB 43.4MB/s 
     |████████████████████████████████| 1.1MB 45.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9509a1b5a62d7560c261bf5de05fd3a1ca1b0ed818e1052e3793927565bd36a5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

Using TensorFlow backend.


In [ ]:
#Reading the csv files
import os
os.chdir('/content/gdrive/My Drive/NLP_Term/Friends')
data = pd.read_csv('Data/friends_train.csv')
pd.set_option('display.max_colwidth',-1)
data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


,idx,sentence,emotion
0,1,also i was the point person on my company ' s transition from the kl - 5 to gr - 6 system .,neutral
1,2,you must ' ve had your hands full .,neutral
2,3,that i did . that i did .,neutral
3,4,so let ' s talk a little bit about your duties .,neutral
4,5,my duties ? all right .,surprise


In [ ]:
sentences = data['sentence']
sentences[:10]

0    also i was the point person on my company ' s transition from the kl - 5 to gr - 6 system .
1    you must ' ve had your hands full .                                                        
2    that i did . that i did .                                                                  
3    so let ' s talk a little bit about your duties .                                           
4    my duties ? all right .                                                                    
5    now you ' ll be heading a whole division , so you ' ll have a lot of duties .              
6    i see .                                                                                    
7    but there ' ll be perhaps 30 people under you so you can dump a certain amount on them .   
8    good to know .                                                                             
9    we can go into detail                                                                      
Name: sentence, dtype: object

In [ ]:
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

["[CLS] also i was the point person on my company ' s transition from the kl - 5 to gr - 6 system . [SEP]",
 "[CLS] you must ' ve had your hands full . [SEP]",
 '[CLS] that i did . that i did . [SEP]',
 "[CLS] so let ' s talk a little bit about your duties . [SEP]",
 '[CLS] my duties ? all right . [SEP]',
 "[CLS] now you ' ll be heading a whole division , so you ' ll have a lot of duties . [SEP]",
 '[CLS] i see . [SEP]',
 "[CLS] but there ' ll be perhaps 30 people under you so you can dump a certain amount on them . [SEP]",
 '[CLS] good to know . [SEP]',
 '[CLS] we can go into detail [SEP]']

In [ ]:
k=0
for i in data['emotion']:
  if i == 'neutral':
    data.at[k, 'emotion'] = 0
  elif i == 'joy':
    data.at[k, 'emotion'] = 1
  elif i == 'sadness':
    data.at[k, 'emotion'] = 2
  elif i == 'anger':
    data.at[k, 'emotion'] = 3
  elif i == 'fear':
    data.at[k, 'emotion'] = 4
  elif i == 'surprise':
    data.at[k, 'emotion'] = 5
  elif i == 'disgust':
    data.at[k, 'emotion'] = 6
  elif i == 'non-neutral':
    data.at[k, 'emotion'] = 7
  k = k + 1
labels = data['emotion'].values
labels

array([0, 0, 0, ..., 5, 0, 7], dtype=object)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] also i was the point person on my company ' s transition from the kl - 5 to gr - 6 system . [SEP]
['[CLS]', 'also', 'i', 'was', 'the', 'point', 'person', 'on', 'my', 'company', "'", 's', 'transition', 'from', 'the', 'kl', '-', '5', 'to', 'gr', '-', '6', 'system', '.', '[SEP]']


In [ ]:
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([  101, 10379,   177, 10134, 10105, 12331, 15042, 10135, 15127,
       12100,   112,   187, 35959, 10188, 10105, 83617,   118,   126,
       10114, 30518,   118,   127, 11787,   119,   102,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [ ]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.2)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.2)

train_labels = train_labels.astype(np.int8)
validation_labels = validation_labels.astype(np.int8)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels, dtype=torch.long)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels, dtype=torch.long)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[:10])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[:10])
print(validation_masks[0])

tensor([  101, 27689, 10112, 19501, 10129,   136,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [ ]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=8)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

epochs = 4

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    print(str(pred_flat) +', '+str(labels_flat))

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def format_time(elapsed):

    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

for epoch_i in range(0, epochs):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()

    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        b_labels = b_labels.long()
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():  
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   100  of    264.    Elapsed: 0:01:19.
  Batch   200  of    264.    Elapsed: 0:02:38.

  Average training loss: 1.37
  Training epcoh took: 0:03:29

Running Validation...
[0 0 0 0 5 5 5 0 5 0 0 5 1 1 0 5 0 5 0 0 0 0 0 0 1 0 7 1 0 1 1 5], [0 5 0 0 5 0 0 0 1 1 0 0 7 7 0 7 0 0 0 0 1 0 0 0 7 0 4 1 0 1 1 7]
[7 0 1 0 0 0 0 0 5 0 1 0 1 0 1 0 0 0 0 0 1 0 1 5 7 5 1 1 0 5 0 0], [1 0 0 0 0 0 5 7 0 0 1 0 0 0 1 7 2 0 0 0 7 0 7 7 0 5 5 1 7 1 0 5]
[0 1 5 0 7 1 0 1 1 5 5 7 0 0 0 0 0 0 7 2 0 0 0 1 5 0 0 0 0 1 0 1], [2 1 5 0 0 1 0 7 7 5 7 3 0 0 0 7 0 5 5 7 0 0 0 1 0 0 0 1 0 1 0 1]
[5 0 0 5 0 0 0 1 1 0 0 0 0 0 5 0 1 5 1 5 0 0 5 0 0 7 0 0 0 0 0 0], [5 7 0 5 7 0 0 1 1 0 0 7 7 0 1 0 7 7 3 5 7 0 3 0 0 7 0 0 0 6 0 0]
[1 1 0 7 5 1 0 0 1 3 0 0 0 0 0 0 0 0 5 5 1 0 0 0 5 0 1 5 5 5 0 0], [1 3 7 3 5 1 3 7 1 3 0 7 5 0 0 0 7 0 5 7 5 0 0 0 7 0 7 5 5 1 0 0]
[0 0 0 0 7 0 0 7 1 0 0 0 0 5 0 1 1 1 0 0 1 0 5 0 1 0 0 0 7 0 0 0], [0 0 0 1 1 0 0 0 2 0 0 7 5 7 7 3 3 7 7 0 0 1 7

In [ ]:
#Loading the test data
#data2 = pd.read_csv('Data/en_data.csv', encoding='CP949')
data2 = pd.read_csv('Data/friends_dev.csv', encoding='CP949')
data2.head()

,idx,sentence,emotion
0,1,"oh my god , he ' s lost it . he ' s totally lost it .",non-neutral
1,2,what ?,surprise
2,3,"or ! or , we could go to the bank , close our accounts and cut them off at the source .",neutral
3,4,you ' re a genius !,joy
4,5,"aww , man , now we won ' t be bank buddies !",sadness


In [ ]:
sentences = data2['sentence']
sentences[:10]

0    oh my god , he ' s lost it . he ' s totally lost it .                                  
1    what ?                                                                                 
2    or ! or , we could go to the bank , close our accounts and cut them off at the source .
3    you ' re a genius !                                                                    
4    aww , man , now we won ' t be bank buddies !                                           
5    now , there ' s two reasons .                                                          
6    hey .                                                                                  
7    hey !                                                                                  
8    ohh , you guys , remember that cute client i told you about ? i bit him .              
9    where !                                                                                
Name: sentence, dtype: object

In [ ]:
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

["[CLS] oh my god , he ' s lost it . he ' s totally lost it . [SEP]",
 '[CLS] what ? [SEP]',
 '[CLS] or ! or , we could go to the bank , close our accounts and cut them off at the source . [SEP]',
 "[CLS] you ' re a genius ! [SEP]",
 "[CLS] aww , man , now we won ' t be bank buddies ! [SEP]",
 "[CLS] now , there ' s two reasons . [SEP]",
 '[CLS] hey . [SEP]',
 '[CLS] hey ! [SEP]',
 '[CLS] ohh , you guys , remember that cute client i told you about ? i bit him . [SEP]',
 '[CLS] where ! [SEP]']

In [ ]:
k=0
for i in data2['emotion']:
  if i == 'neutral':
    data2.at[k, 'emotion'] = 0
  elif i == 'joy':
    data2.at[k, 'emotion'] = 1
  elif i == 'sadness':
    data2.at[k, 'emotion'] = 2
  elif i == 'anger':
    data2.at[k, 'emotion'] = 3
  elif i == 'fear':
    data2.at[k, 'emotion'] = 4
  elif i == 'surprise':
    data2.at[k, 'emotion'] = 5
  elif i == 'disgust':
    data2.at[k, 'emotion'] = 6
  elif i == 'non-neutral':
    data2.at[k, 'emotion'] = 7
  k = k + 1
labels = data2['emotion'].values
labels

array([7, 5, 0, ..., 2, 2, 2], dtype=object)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] oh my god , he ' s lost it . he ' s totally lost it . [SEP]
['[CLS]', 'o', '##h', 'my', 'god', ',', 'he', "'", 's', 'lost', 'it', '.', 'he', "'", 's', 'totally', 'lost', 'it', '.', '[SEP]']


In [ ]:
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,    183,  10237,  15127,  22009,    117,  10261,    112,
          187,  14172,  10271,    119,  10261,    112,    187, 110240,
        14172,  10271,    119,    102,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [ ]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
labels = labels.astype('int8')

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels, dtype=torch.long)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,    183,  10237,  15127,  22009,    117,  10261,    112,    187,
         14172,  10271,    119,  10261,    112,    187, 110240,  14172,  10271,
           119,    102,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [ ]:
batch_size = 32

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
t0 = time.time()

model.eval()

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for step, batch in enumerate(test_dataloader):
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    # print(b_labels)
    
    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

[0 7 0 0 2 5 1 7 0 0 0 5 2 5 7 0 7 0 5 0 7 5 7 0 7 0 0 0 7 7 5 0], [0 0 0 7 2 5 7 0 0 0 0 7 3 7 7 2 0 5 0 7 4 5 7 0 7 0 7 0 6 4 5 0]
[7 1 5 1 1 0 0 7 5 0 0 7 0 0 1 5 0 3 0 5 7 0 0 1 0 5 0 0 1 0 1 0], [1 1 6 4 1 7 5 2 7 7 7 1 7 5 1 5 7 4 0 0 7 7 0 7 0 5 0 0 3 0 1 7]
[5 0 0 0 5 1 1 7 5 7 2 0 0 0 5 0 0 5 5 1 2 5 1 3 7 0 1 0 1 5 0 0], [5 0 0 7 5 1 1 3 1 7 2 3 0 0 5 2 3 5 5 5 7 5 0 6 2 0 0 0 1 5 0 0]
[5 7 0 7 7 0 0 0 0 5 0 1 7 0 0 7 0 5 0 0 0 7 0 7 1 0 0 0 1 7 5 0], [5 3 6 1 4 0 7 3 0 5 3 3 5 0 0 2 0 5 5 0 0 7 7 1 1 0 1 1 7 2 5 7]
[0 7 0 7 7 5 0 0 1 0 0 7 7 1 0 5 7 1 0 1 5 5 7 0 0 0 1 5 0 0 0 1], [0 5 0 3 7 5 0 0 1 0 0 1 7 1 7 5 7 7 3 1 1 5 7 0 4 1 1 5 7 0 0 7]
[5 5 1 7 0 1 0 7 5 7 1 0 7 7 0 0 1 0 0 5 5 0 0 0 0 0 5 0 0 7 5 0], [7 5 5 7 0 7 0 3 5 5 3 7 7 0 0 0 6 0 0 6 5 1 0 7 0 0 5 0 5 7 0 7]
[5 0 0 7 0 5 5 0 0 7 1 1 0 1 0 2 7 5 0 0 0 7 5 1 1 5 0 0 7 0 0 1], [3 0 0 0 0 7 5 7 0 2 1 0 0 7 0 4 7 5 1 7 0 3 7 1 1 5 0 0 7 0 0 1]
[0 7 0 0 5 0 0 0 1 7 0 0 5 1 7 0 7 7 0 5 1 7 7 7 1 5 0 1 7 0 7 3], [0

In [ ]:
output_dir = './model_save/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model.save_pretrained(output_dir)

In [ ]:
filename = './model_save/'
model = BertForSequenceClassification.from_pretrained("./model_save/")

In [ ]:
#Loading the test data
data3 = pd.read_csv('Data/en_data.csv', encoding='CP949')
#data2 = pd.read_csv('Data/friends_test.csv', encoding='CP949')
data3.head()

,id,utterance
0,0,"alright , whadyou do with him ?"
1,1,oh ! you ' re awake !
2,2,then you gotta come clean with ma ! this is not right !
3,3,"yeah , but this is"
4,4,i don ' t wanna hear it ! now go to my room !


In [ ]:
sentences = data3['utterance']
sentences[:50]

0     alright , whadyou do with him ?                                                                                                    
1     oh ! you ' re awake !                                                                                                              
2     then you gotta come clean with ma ! this is not right !                                                                            
3     yeah , but this is                                                                                                                 
4     i don ' t wanna hear it ! now go to my room !                                                                                      
5     i don ' t want him to tell this story for years .                                                                                  
6     oh , but he will . he still tells the story how monica tried to escape from fat camp .                                             
7     i wasn ' t escaping .       

In [ ]:
model.cuda()

conv = { 0: 'neutral', 
         1: 'joy', 
         2: 'sadness', 
         3: 'anger', 
         4: 'fear', 
         5: 'surprise', 
         6: 'disgust', 
         7: 'non-neutral'}
predictions = [] 

sentences = data3['utterance']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []

for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

inputs = torch.tensor(input_ids)
masks = torch.tensor(attention_masks)

batch_size = 32

test_data = TensorDataset(inputs, masks)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
    
t0 = time.time()

model.eval()

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for step, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)

    if step % 50 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))
    
    b_input_ids, b_input_mask = batch
    
    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    logits = np.argmax(logits, axis=1).flatten()
    for i in logits: 
	    predictions.append(conv[i])

#Converting the solution into a CSV file
new_df = pd.DataFrame({"Id": [i for i in range(len(predictions))], "Predicted": predictions})
new_df.to_csv("en_sample.csv", index=False)

[CLS] alright , whadyou do with him ? [SEP]
['[CLS]', 'al', '##right', ',', 'w', '##had', '##yo', '##u', 'do', 'with', 'him', '?', '[SEP]']
  Batch    50  of    103.    Elapsed: 0:00:12.
  Batch   100  of    103.    Elapsed: 0:00:24.
